In [ ]:
# Denna modul är för att testa olika modeller och parametrar 
# för att hitta den bästa modellen för att förutsäga kryptovalutor
# Dessutom testar jag att ta med inflation och guldpriser som features

# Nu används yfinance för att hämta data från Yahoo Finance

# skall flyttas över till my_crypto.py


In [ ]:
import sklearn
import xgboost as xgb
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBClassifier
# from pytrends.request import TrendReq
# import pytrends
import sys
import numpy as np
import pandas as pd
import pickle
from datetime import datetime as dt
import yfinance as yf
import re
import preprocess as pp
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score    
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
# plt.style.use('fivethirtyeight')
import plotly.express as px
import streamlit as st
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

#Import other useful libraries
import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from IPython.display import display
# set display witdh to 200
pd.set_option('display.width', 200)
# set display height to 500
pd.set_option('display.max_rows', 500)
pd.options.display.max_columns = 200    # default 20
pd.options.display.max_colwidth = 60  # default 50
# pd.options.display.precision = 4      # default 6
import logging

# Starta upp logging och inkluder rad nummer
logging.basicConfig(level=logging.INFO, filemode='w', filename='krypto.log', force=True,
                    encoding='utf-8', format= '%(asctime)s - %(levelname)s - %(lineno)d - %(message)s')
logging.info('Startar')


In [ ]:
print('Versioner av softvara som används i denna notebook:')
# print numpy version
print(f'numpy=={np.__version__}')
# print pandas version
print(f'pandas=={pd.__version__}')
# print matplotlib version
print('matplotlib=={}'.format(plt.matplotlib.__version__)) # type: ignore
# print yfinance version
print('yfinance=={}'.format(yf.__version__))
# print pytreands version
print('pytrends==4.8.0  by hand')
# print python-dateutil version
print('python-dateutil==2.8.2  by hand')
# print ta version
print('ta==0.10.2  by hand')
# print streamlit version
print('streamlit=={}'.format(st.__version__))
print('xgboost=={}'.format(xgb.__version__))
print('catboost=={}'.format(catboost.__version__))
print('sklearn=={}'.format(sklearn.__version__))
# print pandas-ta version
print('pandas-ta==0.3.14b0  by hand')
# print plotly version
print('plotly==5.3.1  by hand')
# print pickle version
print('pickle-versionen är samma som python-versionen')
# print python versionen
print('python=={}'.format(sys.version))



För att hämta från yFinance väljer jag att använda följande tickers för learning och testing:  
**Mina egna:**  
['BTC-USD', 'ETH-USD','XRP-USD','BCH-USD','ZRX-USD']  

**Samt 5 till:**  
Binance Coin (BNB-USD)  
Cardano (ADA-USD)  
Dogecoin (DOGE-USD)  
Polkadot (DOT-USD)  
Avalanche (AVAX-USD)  

**Ytterligare 5 som får vara reserv:**  
Solana (SOL-USD)  
Chainlink (LINK-USD)  
Litecoin (LTC-USD)  
Chainlink (LINK-USD)  
Polygon (MATIC-USD)  

In [ ]:
# Dessa 10 kryptovalutor valde jag som input till min ML
tickers = ['BTC-USD','ETH-USD','BCH-USD','XRP-USD','ZRX-USD','BNB-USD','ADA-USD','DOGE-USD','DOT-USD', 'AVAX-USD']
horizons=[2, 5, 60, 250]

## Get inflation data US and SE

In [ ]:
def initiate_data(inflation, df_dates, lang_dict, value_name):
    # display(inflation)
    inflation = inflation.melt(
        id_vars=['Year'], var_name='month', value_name=value_name)

    # use lang_dict to translate month names to numbers
    inflation['month'] = inflation['month'].map(lang_dict)

    inflation['Date'] = pd.to_datetime(inflation['Year'].astype(
        str) + '-' + inflation['month'].astype(str))
    inflation.set_index('Date', inplace=True)
    inflation.drop(['Year', 'month'], axis=1, inplace=True)
    inflation = df_dates.merge(
        inflation, how='left', left_on='Date', right_index=True)
    inflation.set_index('Date', inplace=True)
    inflation[value_name] = inflation[value_name].astype(str)
    inflation[value_name] = inflation[value_name].str.replace(',', '.')
    inflation[value_name] = inflation[value_name].str.replace(
        chr(8209), chr(45))
    inflation[value_name] = inflation[value_name].astype(float)
    inflation[value_name].interpolate(method='linear', inplace=True)
    return inflation

In [ ]:

@st.cache_data
def get_inflation_data(graph=False):
    # Explain this function here        
    # Create a dataframe with dates from 1988-12-01 to today
    df_dates = pd.DataFrame(pd.date_range('1988-12-01', pd.to_datetime('today').date()), columns=['Date'])
    
    US_inflation = pd.read_html('https://www.usinflationcalculator.com/inflation/current-inflation-rates/')
    US_inflation = US_inflation[0]
    # replace the cell including string starting with "Avail" with the NaN
    US_inflation.replace(to_replace=r'^Avail.*$', value=np.nan, regex=True, inplace=True)
    # set the first row as the header and drop the first row
    US_inflation.columns = US_inflation.iloc[0]
    US_inflation.drop(US_inflation.index[0], inplace=True)
    US_inflation.drop('Ave', axis=1, inplace=True)

    # SE_inflation = pd.DataFrame()
    SE_inflation = pd.read_html('https://www.scb.se/hitta-statistik/statistik-efter-amne/priser-och-konsumtion/konsumentprisindex/konsumentprisindex-kpi/pong/tabell-och-diagram/konsumentprisindex-med-fast-ranta-kpif-och-kpif-xe/kpif-12-manadersforandring/')
    SE_inflation = SE_inflation[0]
    SE_inflation.rename(columns={'År': 'Year'}, inplace=True)

    se_dict = dict(Jan='1', Feb='2', Mar='3', Apr='4', Maj='5', Jun='6', Jul='7', Aug='8', Sep='9', Okt='10', Nov='11', Dec='12')
    us_dict = dict(Jan='1', Feb='2', Mar='3', Apr='4', May='5', Jun='6', Jul='7', Aug='8', Sep='9', Oct='10', Nov='11', Dec='12')
    
    SE_inflation = initiate_data(SE_inflation, df_dates, se_dict, value_name='SE_inflation')
    SE_inflation['SE_inflation'] = SE_inflation['SE_inflation'] / 10  # SE_inflation is in percent, divide by 10 to get decimal
    US_inflation = initiate_data(US_inflation, df_dates, us_dict,  value_name='US_inflation')

    if graph:
        # Do a simple plot
        fig, ax = plt.subplots(figsize=(12, 10))
        # set the color of the liens to blue and green
        ax.plot(US_inflation.index, US_inflation['US_inflation'], color='green', label='USA')
        ax.plot(SE_inflation.index, SE_inflation['SE_inflation'], color='blue', label='Sverige')
        
        ax.set_xlabel('Date')
        ax.set_ylabel('Inflation')
        ax.legend()
        plt.show()

    # concat and set one column to US_index and the other to SE_index
    inflations = pd.concat([US_inflation, SE_inflation], axis=1).copy()
    inflations = inflations.dropna()
    # Keep only US_inflation
    inflations = inflations[['US_inflation']]
    return inflations

df_infl = get_inflation_data(graph=True)

In [ ]:
display(list(df_infl.columns))


In [ ]:
# fill up a dataframe with all days from 2005 up to today and set it as index
def get_all_dates():
    start_date = dt(2005, 1, 1)
    end_date = dt.today()
    dates = pd.date_range(start_date, end_date)
    df = pd.DataFrame(index=dates)
    return df


## Get Gold data from yfinance

In [ ]:
def get_gold_data():
    df_dates = pd.DataFrame(pd.date_range(
        '1988-12-01', pd.to_datetime('today').date()), columns=['Date'])
    df_dates.set_index('Date', inplace=True)
    # Hämta historiska guldprisdata (GLD är ticker-symbolen för SPDR Gold Shares ETF)
    gld_data = yf.download('GLD', end=dt.today().date(), progress=False)
    # gld_data.set_index('Date', inplace=True)
    
    # Behåll endast 'Close' priser och döp om kolumnen till 'GLD-USD'
    gld_data = gld_data[['Close']].rename(columns={'Close': 'GLD-USD'})

    df_dates = pd.DataFrame(pd.date_range(start=gld_data.index[0], end=pd.to_datetime('today').date(), freq='D'), columns=['Date']) # type: ignore

    df_dates.set_index('Date', inplace=True)
    gld_data = df_dates.merge(gld_data, how='left', left_on='Date', right_index=True)
    # interpolating missing values
    gld_data.interpolate(method='linear',inplace=True)
    return gld_data

df_gold = get_gold_data()

display(df_gold.shape)
df_gold.plot(figsize=(8, 6), title='Gold Price')
plt.gca().lines[0].set_color("goldenrod")
plt.xlabel('Year')

plt.show()

## Ladda ner all data

In [ ]:
import time
def get_all(tickers):
    # starta timer
    start = time.time()
    # Funktionen används inte just nu    
    df_curr = pd.DataFrame()
    df_vol = pd.DataFrame()
    for ticker in tickers:
        # all_tickers = yf.download(ticker)
        yf_data = yf.download(ticker, period='24mo')
        yf_data = yf_data[['Close', 'Volume']]
        yf_data.columns = [ticker, 'Volume']                   
        df_curr[ticker] = yf_data[ticker]
        df_vol[ticker] = yf_data['Volume']
    df_curr.index = pd.to_datetime(df_curr.index)
    df_vol.index = pd.to_datetime(df_vol.index)
    # skriv ut hur lång tid det tog i sekunder avrundat till 1 decimal
    print(f'Tid för yfinance: {round(time.time() - start, 1)} sekunder')
    return df_curr, df_vol

df_curr, df_vol = get_all(tickers)

# interpolate missing values  
df_curr.interpolate(method='linear',inplace=True)
df_vol.interpolate(method='linear',inplace=True)
print('med yfinance')
display(df_curr.head(3))
display(df_vol.head(3))


In [ ]:
# FOr backtesting
def fit_predict(train, test, target, predictors, model, proba_val=None):
    
    if train[target].nunique() < 2:
        print('kolumner i train',train.columns)
        print('kolumner i test',test.columns)
        print('target',target)
        print('predictors',predictors)
        print('train\n',train[target].value_counts())
        print('test\n',test[target].value_counts())
        print(
            f"Skipping training for target {target} as it has only one unique value in the training data.")
        # stäng ner programmet
        sys.exit()
        
    model.fit(train[predictors], train[target])

    if proba_val:  # använd proba
        preds = model.predict_proba(test[predictors])[:, 1]
        preds[preds >= proba_val] = 1
        preds[preds < proba_val] = 0
    else:    # använd predict utan proba
        preds = model.predict(test[predictors])

    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test[target], preds], axis=1)

    return combined


### Beräkna nya kolumner (predictors)

In [ ]:
df_curr.tail(3)

In [ ]:
### get the data ###
df_curr, df_vol = get_all(tickers)
print('curr:',df_curr.shape)
print('vol:',df_vol.shape)
# inflations = get_inflation_data(graph=False)
print('inflations:',df_infl.shape)
df_gold = get_gold_data()
print('gold:',df_gold.shape)

### Vi gör viss preprocessing för en valuta i taget - nya kolumner

In [ ]:
### preprocessing ###
df_dict = {}
df = pp.preprocessing_currency(df_curr)
if df is not None:
    print('Antal NaN', df.isna().any().sum())

    print()
    df_dict = {}  # en dictionary med samtlig nya dataframes
    for cnt, column_name in enumerate(df.columns):
        df_name = f'df_{column_name}'
        df_dict[df_name] = pp.preprocess(df_curr[[column_name]],df_vol[[column_name]], df_gold, df_infl)
        print(cnt+1, column_name)

display(df_dict.keys())


In [ ]:
df_dict['df_BTC-USD'].head(3)

In [ ]:
the_dict = df_dict.copy()
# Slå ihop alla df_dict till  df2 med gemnsamma kolumner
for df_name in the_dict:
    the_dict[df_name] = the_dict[df_name].rename(
        columns={the_dict[df_name].columns[0]: 'Close'})
    # reset index in df_dict[df_name] but keep Date column
    the_dict[df_name] = the_dict[df_name].reset_index(drop=False)
    # add Ticker column to df_dict[df_name]
    the_dict[df_name]['Ticker'] = df_name

df2 = pd.concat(the_dict.values(), axis=0)
df2.sort_values(by=['Date'], inplace=True)
df2.query('Date == "2023-05-01" and Ticker == "df_DOGE-USD"')
df2.set_index('Date', inplace=True)
display(df2.head(3))
display(df2.tail(3))
display(df2.columns)


In [ ]:


# compute Tommorow's price to be Close next day
df2['Tomorrow'] = df2.groupby('Ticker')['Close'].shift(-1)

# compute the percentage change
df2['Percentage Change'] = (df2['Tomorrow'] - df2['Close']) / df2['Close'] * 100

df2['y1'] = (df2['Tomorrow'] > df2['Close']).astype(int)
df2.dropna(inplace=True)

### Preprocessing för alla valutor tillsammans

In [ ]:
predictors = ['Close','Ratio_2', 'Trend_2', 'Ratio_5', 'Trend_5', 'Ratio_30', 'Trend_30', 'Ratio_60', 'Trend_60', 'Ratio_90', 'Trend_90',
                  'Ratio_250', 'Trend_250', 'GLD-USD', 'GLD_Ratio_2', 'GLD_Ratio_5', 'GLD_Ratio_30', 'GLD_Ratio_60', 'GLD_Ratio_90', 'GLD_Ratio_250',
                  'Volume', 'vol_Ratio_2', 'vol_Ratio_5', 'vol_Ratio_30', 'vol_Ratio_60', 'vol_Ratio_90', 'vol_Ratio_250',
                  'US_inflation', 'infl_Ratio_75', 'infl_Ratio_90', 'infl_Ratio_250', 'diff', 'before_kvot','before_up']
    


In [ ]:
print(len(df2.columns))
len(predictors)
# skillnaden mellan columns och predictors är 'y1', 'Tomorrow', 'Percentage Change'
predictors_diff = [col for col in df2.columns if col not in predictors]
predictors_diff

In [ ]:

tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
print(predictors+['y1'])
X = df2[predictors].dropna()
y = df2['y1']
print(X.shape)
print(y.shape)

#### Välj mellan GridSearch och RandomSearch

In [ ]:
randomsearch = True   # RandmSearchCV för de två tunga modellerna


def combinations(param_list):
    total_combinations = 1
    for key in param_list:
        if key.startswith('classifier__'):
            total_combinations *= len(param_list[key])
    return total_combinations


In [ ]:
# Initialize the hyperparameters for each dictionary
# each having a key as ‘classifier’ and value as estimator object.
#The hyperparameter keys should start with the word classifier separated
# by ‘__’ (double underscore)

# Define parameters for CatBoostClassifier
paramCat = {}
paramCat['classifier__iterations'] = [75, 100, 250, 300, 325, 350]
paramCat['classifier__depth'] = [ 3, 4, 5, 7, 9]
# paramCat['classifier__class_weights'] = [{0: 1, 1: 1}, {0: 1, 1: 2} ]
paramCat['classifier__learning_rate'] = [0.01, 0.03, 0.05, 0.07, 0.09]
paramCat['classifier'] = [CatBoostClassifier(random_state=42, verbose=False)]
print('Total combinations for CatBoostClassifier = ', combinations(paramCat))

# Define parameters for XGBClassifier
paramXGB = {}
paramXGB['classifier__n_estimators'] = [75, 100, 250,300, 350]
paramXGB['classifier__max_depth'] = [2, 3, 5, 10, 12]
paramXGB['classifier__learning_rate'] = [0.01, 0.03, 0.05, 0.07, 0.09]
# paramXGB['classifier__scale_pos_weight'] = [1, 5, 10]
paramXGB['classifier'] = [XGBClassifier(
    random_state=42, eval_metric='logloss')]
print('Total combinations for XGBClassifier = ', combinations(paramXGB))


# Define parameters for Random Forest
paramRF = {}
paramRF['classifier__n_estimators'] = [75, 100, 200, 350, 400, 500]
paramRF['classifier__min_samples_split'] = [2, 5, 10, 12]
paramRF['classifier__max_depth'] = [ 5, 10, 12, 14]
paramRF['classifier__min_samples_leaf'] = [1,2,4]
paramRF['classifier__max_features'] = ['auto', 'sqrt']
paramRF['classifier__bootstrap'] = [True, False]
paramRF['classifier'] = [RandomForestClassifier(random_state=42)]
print('Total combinations for RandomForestClassifier = ', combinations(paramRF))

# Define parameters for support vector machine (SVC)
paramSVC = {}
paramSVC['classifier__kernel'] = ['linear', 'rbf']
paramSVC['classifier__C'] = [0.01, 1, 10, 100]
paramSVC['classifier__gamma'] = ['scale', 'auto', 0.1, 1, 10]
paramSVC['classifier__shrinking'] = [True, False]
paramSVC['classifier'] = [SVC(random_state=42)]
print('Total combinations for SVC = ', combinations(paramSVC))

# Define parameters for Logistic regression
paramLR = {}
paramLR['classifier__C'] = [0.01, 0.1, 1, 10, 20, 50]
paramLR['classifier__penalty'] = ['l1', 'l2']
paramLR['classifier__solver'] = ['liblinear', 'saga']  # or 'saga'
paramLR['classifier__class_weight'] = [None]
paramLR['classifier'] = [LogisticRegression(random_state=42)]
print('Total combinations for LogisticRegression = ', combinations(paramLR))

# Define parameters for K neighbors classifier
paramKN = {}
paramKN['classifier__n_neighbors'] = [ 7, 10, 20, 30, 50,60]
paramKN['classifier__weights'] = ['uniform', 'distance']
paramKN['classifier__algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
paramKN['classifier__leaf_size'] = [10, 30, 50, 70, 90]
paramKN['classifier'] = [KNeighborsClassifier()]
print('Total combinations for KNeighborsClassifier = ', combinations(paramKN))

# Define parameters for Gradient boosting
paramGBC = {}
paramGBC['classifier__max_depth'] = [3, 5, 10, 20]
paramGBC['classifier__min_samples_leaf'] = [1, 2, 4]
paramGBC['classifier__min_samples_split'] = [ 5, 10, 12]
paramGBC['classifier__n_estimators'] = [10, 50, 100, 200, 300]

# paramGBC['classifier__max_features'] = ['auto', 'sqrt', 'log2', None]
# paramGBC['classifier__subsample'] = [0.5, 0.8, 1.0]
# paramGBC['classifier__loss'] = ['deviance', 'exponential']

paramGBC['classifier'] = [GradientBoostingClassifier(random_state=42)]
print('Total combinations for GradientBoostingClassifier = ', combinations(paramGBC))

# define the pipeline to include scaling and the model.
# Prepare the pipeline for the 1st model, others will be loaded appropriately
#during the Grid Search
#This pipeline will be the input to cross_val_score, instead of the model.
steps = list()
steps.append(('scaler', StandardScaler()))
steps.append(('classifier', RandomForestClassifier(random_state=42)))
pipeline = Pipeline(steps=steps)

#Capture all parameter dictionaries as a list
params = [paramSVC, paramLR, paramKN, paramGBC]
# Total parameters for all 5 models = 27+20+48+5+9 = 109

### GridSearchCV för lätta modeller

In [ ]:
# take the time
start_time = time.time()
#Grid search - including cross validation
grid = GridSearchCV(pipeline, params, cv=tscv, n_jobs=-
                    1, scoring='roc_auc').fit(X, y)

#Gridsearch object (in our case 'grid') stores all the information about
#the best model and corresponding hyperparameters.
# print the best parameters...
print(grid.best_params_)

# print best score for the best model (in our case roc_auc score)
print(grid.best_score_)

# Stats for each test - we have a total 125 tests
means = grid.cv_results_['mean_test_score']
params_summary = grid.cv_results_['params']
# print the time taken
print("--- %s seconds ---" % round((time.time() - start_time),1))

In [ ]:

#Capture all data into a Data Frame
df_res1 = pd.DataFrame(list(zip(means, params_summary)),
                  columns=['Mean Score', 'Parms']).sort_values(by=['Mean Score'], ascending=False)

In [ ]:
df_res1 = df_res1.sort_values(by='Mean Score', ascending=False)
df_res1.to_csv('GridSearchResults1.csv', index=False)

In [ ]:
df_res1 = pd.read_csv('GridSearchResults1.csv')

df_res1.head()

### RandomizedSearchCV av de tre tunga modellerna

In [ ]:
if randomsearch:
    from sklearn.model_selection import RandomizedSearchCV

    # Add the new parameter dictionaries to the list
    params = [paramCat, paramXGB, paramRF]

    # Number of iterations for randomized search
    n_iter_search = 50

    #### Random search RF - including cross validation ####
    # Take the time
    start_time = time.time()
    randomRF = RandomizedSearchCV(
        pipeline, param_distributions=params[2], n_iter=n_iter_search, cv=tscv, n_jobs=-1, scoring='roc_auc', random_state=42).fit(X, y)

    # print the best parameters...
    print(randomRF.best_params_)

    # print best score for the best model (in our case roc_auc score)
    print(randomRF.best_score_)

    # Stats for each test - we have a total 48 tests
    meansRF = randomRF.cv_results_['mean_test_score']
    params_summaryRF = randomRF.cv_results_['params']

    #Capture all data into a Data Frame
    df_resRF = pd.DataFrame(list(zip(meansRF, params_summaryRF)),
                            columns=['Mean Score', 'Parms'])

    # Take the time
    print("--- The RF time was %s seconds ---" %round((time.time() - start_time), 1))
    df_resRF = df_resRF.sort_values(by='Mean Score', ascending=False)
    df_resRF.to_csv('RandomSearchResultsRF.csv', index=False)

    display(df_resRF.head())
    
    
    #### Random search XGB - including cross validation ####
    # Take the time
    start_time = time.time()
    randomXGB = RandomizedSearchCV(
        pipeline, param_distributions=params[1], n_iter=n_iter_search, cv=tscv, n_jobs=-1, scoring='roc_auc', random_state=42).fit(X, y)

    # print the best parameters...
    print(randomXGB.best_params_)

    # print best score for the best model (in our case roc_auc score)
    print(randomXGB.best_score_)

    # Stats for each test - we have a total 48 tests
    meansXGB = randomXGB.cv_results_['mean_test_score']
    params_summaryXGB = randomXGB.cv_results_['params']

    #Capture all data into a Data Frame
    df_resXGB = pd.DataFrame(list(zip(meansXGB, params_summaryXGB)),
                            columns=['Mean Score', 'Parms'])

    # Take the time
    print("--- The XGB time was %s seconds ---" % round((time.time() - start_time), 1))
    df_resXGB = df_resXGB.sort_values(by='Mean Score', ascending=False)
    df_resXGB.to_csv('RandomSearchResultsXGB.csv', index=False)

    display(df_resXGB.head())

    #### Random search CATB - including cross validation ####
    # Take the time
    start_time = time.time()
    randomCAT = RandomizedSearchCV(
        pipeline, param_distributions=params[0], n_iter=n_iter_search, cv=tscv, n_jobs=-1, scoring='roc_auc', random_state=42).fit(X, y)

    # print the best parameters...
    print(randomCAT.best_params_)

    # print best score for the best model (in our case roc_auc score)
    print(randomCAT.best_score_)

    # Stats for each test - we have a total 144 tests
    mensCAT = randomCAT.cv_results_['mean_test_score']
    params_summaryCAT = randomCAT.cv_results_['params']

    #Capture all data into a Data Frame
    df_resCAT = pd.DataFrame(list(zip(mensCAT, params_summaryCAT)),
                            columns=['Mean Score', 'Parms'])
    df_resCAT =df_resCAT.sort_values(by='Mean Score', ascending=False)  
    df_resCAT.to_csv('RandomSearchResultsCAT.csv', index=False)

    display(df_resCAT.head())
    # Take the time
    print("--- The Cat time was %s seconds ---" % round((time.time() - start_time), 1))


### GridSearchCV även för de tunga

In [ ]:
if not randomsearch:
    # Make GridSearch one by one for each model - because it takes so long time
    params = [paramCat, paramXGB, paramRF]

    #### search XGB ####
    start_time = time.time()
    grid2a = GridSearchCV(pipeline, params, cv=tscv, n_jobs=-
                        1, scoring='roc_auc').fit(X, y)

    # print the best parameters...
    print(grid2a.best_params_)

    # print best score for the best model (in our case roc_auc score)
    print(grid2a.best_score_)

    # Stats for each test - we have a total 48 tests
    means2a = grid2a.cv_results_['mean_test_score']
    params_summary2a = grid2a.cv_results_['params']

    #Capture all data into a Data Frame
    df_resRF = pd.DataFrame(list(zip(means2a, params_summary2a)),
                    columns=['Mean Score', 'Parms'])

    df_resRF.to_csv('GridSearchResults2a.csv', index=False)

    # Take the time
    print("--- The time was %s seconds ---" % round((time.time() - start_time),1))
    
    df_resRF = pd.read_csv('GridSearchResults2a.csv').sort_values(
        by='Mean Score', ascending=False)

    df_resRF.head()


In [ ]:
if not randomsearch:
    #### Grid search CATB ####
    # Take the time
    start_time = time.time()
    grid2b = GridSearchCV(pipeline, params[:1], cv=tscv, n_jobs=-
                        1, scoring='roc_auc').fit(X, y)
    # print the best parameters...
    print(grid2b.best_params_)

    # print best score for the best model (in our case roc_auc score)
    print(grid2b.best_score_)

    # Stats for each test - we have a total 144 tests
    meansXGB = grid2b.cv_results_['mean_test_score']
    params_summaryXGB = grid2b.cv_results_['params']

    #Capture all data into a Data Frame
    df_resXGB = pd.DataFrame(list(zip(meansXGB, params_summaryXGB)), # type: ignore
                            columns=['Mean Score', 'Parms'])

    df_resXGB.to_csv('GridSearchResults2b.csv', index=False)

    # Take the time
    print("--- The time was %s seconds ---" % round((time.time() - start_time),1))

### Summera alla modellers performance

In [ ]:
df_res1 = pd.read_csv('GridSearchResults1.csv')

if randomsearch:
    df_resRF = pd.read_csv('RandomSearchResultsRF.csv')
    df_resRF['Model'] = 'RandomForrestClassifier'
    df_resXGB = pd.read_csv('RandomSearchResultsXGB.csv')
    df_resXGB['Model'] = 'XGBoost_Classifier'
    df_resCAT = pd.read_csv('RandomSearchResultsCAT.csv')
    df_resCAT['Model'] = 'CatBoostClassifier'
else:
    df_resRF = pd.read_csv('GridSearchResults2a.csv')
    df_resRF['Model'] = 'WARNING change this'
    df_resXGB = pd.read_csv('GridSearchResults2b.csv')
    df_resXGB['Model'] = 'WARNING change this'
    df_resCAT = pd.DataFrame()

df_res = pd.concat([df_res1, df_resRF, df_resXGB, df_resCAT], ignore_index=True).sort_values(
    by='Mean Score', ascending=False)
df_res.to_csv('GridSearchResults.csv', index=False)


def extract_model_and_params(row):
    model_regex = r'\'classifier\':\s*([^,]+)'
    param_regex = r'\'classifier__([a-zA-Z0-9_]+)\':\s*([^,}]+)'

    model_match = re.search(model_regex, row['Parms'])
    model = model_match.group(1) if model_match else 'Unknown'
    # print(type(row.Model))
    model = 'CatBoostClassifier' if 'CatBoostClassifier' == row.Model else model
    model = 'XGBoost_Classifier' if 'XGBoost_Classifier' == row.Model else model
    model = 'RandomForrestClassifier' if 'RandomForrestClassifier' == row.Model else model
    # Ersätt objektreferenser med klassnamn
    model = re.sub(r'<[^>]*>', lambda x: 'CatBoostClassifier' if 'catboost' in x.group(0)
                   else x.group(0).split('.')[1], model)
    # print(model)
    # Ta bort allt efter och inklusive '('
    model = re.sub(r'\(.*|\}.*', '', model).strip()

    params = re.findall(param_regex, row['Parms'])
    params_clean = {k: v.strip() for k, v in params}
    # print(params_clean)
    return pd.Series([model, params_clean])

print(df_res.columns)
print(df_res.Model.unique())


In [ ]:

# Skapa nya kolumner för model och parametrar utan 'classifier__' prefix

df_res[['Model', 'Parameters_clean']] = df_res.apply(extract_model_and_params, axis=1)
df_res.sort_values(by='Mean Score', ascending=False, inplace=True)
# Ta bort den gamla 'Parameters' kolumnen
df_res = df_res.drop(columns=['Parms'])

print(
    f'The winner is {df_res.iloc[0].Model} with params\n {df_res.iloc[0]["Parameters_clean"]}')

print()
pd.set_option('display.max_colwidth', None)
print('De bästa resultaten per modell sorterade efter Mean Score')
m = df_res.groupby('Model').head(1).sort_values(by='Mean Score', ascending=False)
print(m[['Model', 'Mean Score', 'Parameters_clean']].to_string(index=False))


In [ ]:

"""Det nyaste resultatet (24/5)   
CatBoostClassifier          0.561043 {'learning_rate': '0.01', 'iterations': '350', 'depth': '9'}  
RandomForrestClassifier     0.559331 {'n_estimators': '100', 'min_samples_split': '2', 'min_samples_leaf': '1', 'max_features': ''auto'', 'max_depth': '12', 'bootstrap': 'True'}  
GradientBoostingClassifier  0.555753 {'max_depth': '20', 'min_samples_leaf': '2', 'min_samples_split': '10', 'n_estimators': '100'}  
SVC                         0.552393 {'C': '1', 'gamma': ''scale'', 'kernel': ''rbf'', 'shrinking': 'False'}    
LogisticRegression          0.551225 {'C': '0.1', 'class_weight': 'None', 'penalty': ''l1'', 'solver': ''saga''}  
XGBoost_Classifier          0.550834 {'n_estimators': '300', 'max_depth': '12', 'learning_rate': '0.05'}  
KNeighborsClassifier        0.547992 {'algorithm': ''kd_tree'', 'leaf_size': '90', 'n_neighbors': '50', 'weights': ''uniform''}  

    resultatet (23/5)   
RandomForrestClassifier     0.561302 {'n_estimators': ures': ''auto'', 'max_depth': '14', 'bootstrap': 'True'}  
CatBoostClassifier          0.556916 {'learning_rate': '0.01', 'iterations': '325', 'depth': '7'}  
GradientBoostingClassifier  0.553260 {'max_depth': '10', 'min_samples_leaf': '1', 'min_samples_split': '10', 'n_estimators': '50'}  
LogisticRegression          0.548242 {'C': '0.1', 'class_weight': 'None', 'penalty': ''l1'', 'solver': ''saga''}  
SVC                         0.546807 {'C': '1', 'gamma': ''scale'', 'kernel': ''rbf'', 'shrinking': 'True'}  
KNeighborsClassifier        0.544760 {'algorithm': ''auto'', 'leaf_size': '10', 'n_neighbors': '60', 'weights': ''uniform''}  
XGBoost_Classifier          0.539619 {'n_estimators': '100', 'max_depth': '3', 'learning_rate': '0.07'}  


"""


In [ ]:
""" Det äldre resultatet                                                                                                                              Parameters_clean
XGBoost_Classifier      0.557098 {'n_estimators': '250', 'max_depth': '12', 'learning_rate': '0.09'}
GradientBoostingClassif 0.552184 {'max_depth': '10', 'min_samples_leaf': '4', 'min_samples_split': '12', 'n_estimators': '300'}
CatBoostClassifier      0.551837 {'learning_rate': '0.09', 'iterations': '300', 'depth': '2'}
RandomForrestClassifier 0.548157 {'n_estimators': '350', 'min_samples_split': '5', 'min_samples_leaf': '4', 'max_features': ''auto'', 'max_depth': '10', 'bootstrap': 'False'}
SVC                     0.540441 {'C': '1', 'gamma': ''scale'', 'kernel': ''rbf'', 'shrinking': 'True'}
LogisticRegression      0.528314 {'C': '50', 'class_weight': 'None', 'penalty': ''l2'', 'solver': ''liblinear''}
KNeighborsClassifier    0.517581 {'algorithm': ''ball_tree'', 'leaf_size': '10', 'n_neighbors': '50', 'weights': ''distance''}
"""

### Skapa en modell av den bästa ovan

In [ ]:
best_model = 'CB'
# Modellen baseras på mina förvalda tickers
print(tickers)

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [ ]:

def XGB(X, y):
    import xgboost
    from xgboost import XGBClassifier
    # Skapa en StandardScaler
    scaler = StandardScaler()

    # Förbered en pipeline som först skalar datan och sedan kör XGBoost
    XGB_dict = {'n_estimators': 250, 'max_depth': 12, 'learning_rate': 0.09}
    model = Pipeline([
        ('scaler', scaler),
        ('classifier', XGBClassifier(**XGB_dict))
    ])

    # Träna modellen med datan X och målvariabeln y
    model.fit(X, y)
    print(f'xgboost version: {xgboost.__version__}')
    return model



In [ ]:
def RF(X, y):
    from sklearn.ensemble import RandomForestClassifier
    # Skapa en StandardScaler
    scaler = StandardScaler()

    # Förbered en pipeline som först skalar datan och sedan kör RF
    rf_dict={'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 14, 'bootstrap': True}
    model = Pipeline([
        ('scaler', scaler),
        ('classifier', RandomForestClassifier(**rf_dict))
    ])

    # Träna modellen med datan X och målvariabeln y
    model.fit(X, y)

    # print randomforest version
    print(f'randomforest version: {sklearn.__version__}')

    return model


def CB(X, y):
    from catboost import CatBoostClassifier
    # Skapa en StandardScaler
    scaler = StandardScaler()

    # Förbered en pipeline som först skalar datan och sedan kör CatBoost
    cb_dict = {'learning_rate': 0.01, 'iterations': 350, 'depth': 9}
    model = Pipeline([
        ('scaler', scaler),
        ('classifier', CatBoostClassifier(**cb_dict))
    ])
    
    
    # Träna modellen med datan X och målvariabeln y
    model.fit(X, y)

    # print catboost version
    print(f'catboost version: {catboost.__version__}')

    return model


In [ ]:

if best_model == 'RF':
    my_model = RF(X, y)
elif best_model == 'XGB':    
    my_model = XGB(X, y)
elif best_model == 'CB':    
    my_model = CB(X, y)    
else:
    print('No model selected')
    sys.exit()

print(f'sklearn version: {sklearn.__version__}')
# save my_model
pickle.dump(my_model, open("my_model.pkl", "wb"))

# load my_model
my_model = pickle.load(open("my_model.pkl", "rb"))  


In [ ]:

np.sum((my_model.predict(X) == y))/len(y)
# my_model.get_params()

# Tester